In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()

spark= SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
loan_schema = """loan_id string, member_id string, loan_amount float, 
funded_amount float, loan_term_months string, interest_rate float, 
monthly_installment float, issue_date string, loan_status string, 
loan_purpose string, loan_title string"""

In [3]:
loans_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loan_schema) \
.load("/user/itv015703/lendingclubproject/raw/loans_data_csv")

In [4]:
loans_raw_df 

loan_id,member_id,loan_amount,funded_amount,loan_term_months,interest_rate,monthly_installment,issue_date,loan_status,loan_purpose,loan_title
94761152,1ae08eb0c074de6fb...,26975.0,26975.0,60 months,30.84,886.71,Dec-2016,Fully Paid,debt_consolidation,Debt consolidation
94093520,82ae44edab97e1c95...,28000.0,28000.0,60 months,7.99,567.61,Dec-2016,Current,credit_card,Credit card refin...
95126205,7a92464a7eef5fa98...,12000.0,12000.0,36 months,12.74,402.83,Dec-2016,Current,debt_consolidation,Debt consolidation
95217505,6fc2a824c46b88831...,9600.0,9600.0,36 months,13.99,328.06,Dec-2016,Late (31-120 days),debt_consolidation,Debt consolidation
95096173,d72283a720a1bb13c...,2700.0,2700.0,36 months,8.24,84.91,Dec-2016,Fully Paid,credit_card,Credit card refin...
95148408,0b0616b87c7476b53...,22400.0,22400.0,36 months,7.24,694.11,Dec-2016,Fully Paid,debt_consolidation,Debt consolidation
93793227,b89d45166d32ba422...,12000.0,12000.0,36 months,15.99,421.83,Dec-2016,Fully Paid,credit_card,Credit card refin...
94566736,4f170572ef915ba17...,35000.0,35000.0,60 months,13.99,814.21,Dec-2016,Current,debt_consolidation,Debt consolidation
95256545,30ca62c00e346e480...,5600.0,5600.0,36 months,13.99,191.37,Dec-2016,Current,other,Other
95272111,64f06e51bc16579dd...,5000.0,5000.0,36 months,7.99,156.66,Dec-2016,Fully Paid,medical,Medical expenses


In [5]:
loans_raw_df .printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amount: float (nullable = true)
 |-- funded_amount: float (nullable = true)
 |-- loan_term_months: string (nullable = true)
 |-- interest_rate: float (nullable = true)
 |-- monthly_installment: float (nullable = true)
 |-- issue_date: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- loan_purpose: string (nullable = true)
 |-- loan_title: string (nullable = true)



In [6]:
from pyspark.sql.functions import current_timestamp

loans_ingested_df = loans_raw_df.withColumn("ingest_date", current_timestamp())

In [7]:
loans_ingested_df

loan_id,member_id,loan_amount,funded_amount,loan_term_months,interest_rate,monthly_installment,issue_date,loan_status,loan_purpose,loan_title,ingest_date
94761152,1ae08eb0c074de6fb...,26975.0,26975.0,60 months,30.84,886.71,Dec-2016,Fully Paid,debt_consolidation,Debt consolidation,2025-03-30 06:49:...
94093520,82ae44edab97e1c95...,28000.0,28000.0,60 months,7.99,567.61,Dec-2016,Current,credit_card,Credit card refin...,2025-03-30 06:49:...
95126205,7a92464a7eef5fa98...,12000.0,12000.0,36 months,12.74,402.83,Dec-2016,Current,debt_consolidation,Debt consolidation,2025-03-30 06:49:...
95217505,6fc2a824c46b88831...,9600.0,9600.0,36 months,13.99,328.06,Dec-2016,Late (31-120 days),debt_consolidation,Debt consolidation,2025-03-30 06:49:...
95096173,d72283a720a1bb13c...,2700.0,2700.0,36 months,8.24,84.91,Dec-2016,Fully Paid,credit_card,Credit card refin...,2025-03-30 06:49:...
95148408,0b0616b87c7476b53...,22400.0,22400.0,36 months,7.24,694.11,Dec-2016,Fully Paid,debt_consolidation,Debt consolidation,2025-03-30 06:49:...
93793227,b89d45166d32ba422...,12000.0,12000.0,36 months,15.99,421.83,Dec-2016,Fully Paid,credit_card,Credit card refin...,2025-03-30 06:49:...
94566736,4f170572ef915ba17...,35000.0,35000.0,60 months,13.99,814.21,Dec-2016,Current,debt_consolidation,Debt consolidation,2025-03-30 06:49:...
95256545,30ca62c00e346e480...,5600.0,5600.0,36 months,13.99,191.37,Dec-2016,Current,other,Other,2025-03-30 06:49:...
95272111,64f06e51bc16579dd...,5000.0,5000.0,36 months,7.99,156.66,Dec-2016,Fully Paid,medical,Medical expenses,2025-03-30 06:49:...


In [8]:
loans_ingested_df.createOrReplaceTempView("loans")

In [9]:
spark.sql("select * from loans where loan_amount is null")

loan_id,member_id,loan_amount,funded_amount,loan_term_months,interest_rate,monthly_installment,issue_date,loan_status,loan_purpose,loan_title,ingest_date
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2025-03-30 06:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2025-03-30 06:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2025-03-30 06:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2025-03-30 06:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2025-03-30 06:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2025-03-30 06:49:...
Loans that do not...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2025-03-30 06:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2025-03-30 06:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2025-03-30 06:49:...
Total amount fund...,e3b0c44298fc1c149...,null,null,null,null,null,null,null,null,null,2025-03-30 06:49:...


In [10]:
columns_to_check = ["loan_amount", "funded_amount", "loan_term_months", "interest_rate", "monthly_installment", "issue_date", "loan_status", "loan_purpose"]

In [11]:
loans_filtered_df = loans_ingested_df.na.drop(subset=columns_to_check)

In [12]:
loans_filtered_df

loan_id,member_id,loan_amount,funded_amount,loan_term_months,interest_rate,monthly_installment,issue_date,loan_status,loan_purpose,loan_title,ingest_date
94761152,1ae08eb0c074de6fb...,26975.0,26975.0,60 months,30.84,886.71,Dec-2016,Fully Paid,debt_consolidation,Debt consolidation,2025-03-30 06:49:...
94093520,82ae44edab97e1c95...,28000.0,28000.0,60 months,7.99,567.61,Dec-2016,Current,credit_card,Credit card refin...,2025-03-30 06:49:...
95126205,7a92464a7eef5fa98...,12000.0,12000.0,36 months,12.74,402.83,Dec-2016,Current,debt_consolidation,Debt consolidation,2025-03-30 06:49:...
95217505,6fc2a824c46b88831...,9600.0,9600.0,36 months,13.99,328.06,Dec-2016,Late (31-120 days),debt_consolidation,Debt consolidation,2025-03-30 06:49:...
95096173,d72283a720a1bb13c...,2700.0,2700.0,36 months,8.24,84.91,Dec-2016,Fully Paid,credit_card,Credit card refin...,2025-03-30 06:49:...
95148408,0b0616b87c7476b53...,22400.0,22400.0,36 months,7.24,694.11,Dec-2016,Fully Paid,debt_consolidation,Debt consolidation,2025-03-30 06:49:...
93793227,b89d45166d32ba422...,12000.0,12000.0,36 months,15.99,421.83,Dec-2016,Fully Paid,credit_card,Credit card refin...,2025-03-30 06:49:...
94566736,4f170572ef915ba17...,35000.0,35000.0,60 months,13.99,814.21,Dec-2016,Current,debt_consolidation,Debt consolidation,2025-03-30 06:49:...
95256545,30ca62c00e346e480...,5600.0,5600.0,36 months,13.99,191.37,Dec-2016,Current,other,Other,2025-03-30 06:49:...
95272111,64f06e51bc16579dd...,5000.0,5000.0,36 months,7.99,156.66,Dec-2016,Fully Paid,medical,Medical expenses,2025-03-30 06:49:...


In [13]:
loans_filtered_df.createOrReplaceTempView("loans")

In [14]:
from pyspark.sql.functions import regexp_replace, col
loans_terms_modified= loans_filtered_df.withColumn("loan_term_months", (regexp_replace(col("loan_term_months"), " months", "") \
                                         .cast("int")/12) \
                                         .cast("int")) \
                                         .withColumnRenamed("loan_term_months", "loan_term_years")

In [15]:
loans_terms_modified.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amount: float (nullable = true)
 |-- funded_amount: float (nullable = true)
 |-- loan_term_years: integer (nullable = true)
 |-- interest_rate: float (nullable = true)
 |-- monthly_installment: float (nullable = true)
 |-- issue_date: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- loan_purpose: string (nullable = true)
 |-- loan_title: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [16]:
loans_terms_modified.createOrReplaceTempView("loans")

In [17]:
spark.sql("select distinct(loan_purpose) from loans")

loan_purpose
"guaranteed!"""
and if they are a...
never had any tro...
<br/><br/>Lending...
Bank of America c...
stocks
please feel free ...
I became his prim...
brakes
on one of the bus...


In [18]:
spark.sql("select loan_purpose, count(*) as total from loans group by loan_purpose order by total desc")

loan_purpose,total
debt_consolidation,1277790
credit_card,516926
home_improvement,150440
other,139413
major_purchase,50429
medical,27481
small_business,24659
car,24009
vacation,15525
moving,15402


In [19]:
loan_purpose_lookup = ["debt_consolidation", "credit_card", "home_improvement", "other", "major_purchase", "medical", "small_business", "car", "vacation", "moving", "house", "wedding", "renewable_energy", "educational"]

In [20]:
from pyspark.sql.functions import when, col

loan_purpose_modified_df = loans_terms_modified.withColumn("loan_purpose", \
                                when(col("loan_purpose").isin(loan_purpose_lookup), col("loan_purpose")).\
                                otherwise("other"))

In [38]:
loan_purpose_modified_df.createOrReplaceTempView("loans")

In [39]:
spark.sql("select loan_purpose, count(*) as total from loans group by loan_purpose order by total desc")

Py4JJavaError: An error occurred while calling o242.showString.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(loan_purpose#773, 200), ENSURE_REQUIREMENTS, [id=#298]
+- *(1) HashAggregate(keys=[loan_purpose#773], functions=[partial_count(1)], output=[loan_purpose#773, count#810L])
   +- *(1) Project [CASE WHEN loan_purpose#9 INSET (home_improvement,medical,credit_card,other,moving,renewable_energy,vacation,small_business,house,debt_consolidation,educational,wedding,major_purchase,car) THEN loan_purpose#9 ELSE other END AS loan_purpose#773]
      +- *(1) Filter AtLeastNNulls(n, loan_amount#2,funded_amount#3,loan_term_months#4,interest_rate#5,monthly_installment#6,issue_date#7,loan_status#8,loan_purpose#9)
         +- FileScan csv [loan_amount#2,funded_amount#3,loan_term_months#4,interest_rate#5,monthly_installment#6,issue_date#7,loan_status#8,loan_purpose#9] Batched: false, DataFilters: [AtLeastNNulls(n, loan_amount#2,funded_am

Py4JJavaError: An error occurred while calling o242.getRowsToPython.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(loan_purpose#773, 200), ENSURE_REQUIREMENTS, [id=#328]
+- *(1) HashAggregate(keys=[loan_purpose#773], functions=[partial_count(1)], output=[loan_purpose#773, count#810L])
   +- *(1) Project [CASE WHEN loan_purpose#9 INSET (home_improvement,medical,credit_card,other,moving,renewable_energy,vacation,small_business,house,debt_consolidation,educational,wedding,major_purchase,car) THEN loan_purpose#9 ELSE other END AS loan_purpose#773]
      +- *(1) Filter AtLeastNNulls(n, loan_amount#2,funded_amount#3,loan_term_months#4,interest_rate#5,monthly_installment#6,issue_date#7,loan_status#8,loan_purpose#9)
         +- FileScan csv [loan_amount#2,funded_amount#3,loan_term_months#4,interest_rate#5,monthly_installment#6,issue_date#7,loan_status#8,loan_purpose#9] Batched: false, DataFilters: [AtLeastNNulls(n, loan_amount#2,fund